<a href="https://colab.research.google.com/github/Takumi173/DifyApps/blob/main/Dataset_JSON_Reviewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

## 処理用にデータを結合

In [1]:
# データのコピー
!git clone https://github.com/cdisc-org/sdtm-adam-pilot-project.git

Cloning into 'sdtm-adam-pilot-project'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 224 (delta 64), reused 220 (delta 61), pack-reused 0 (from 0)
Receiving objects: 100% (224/224), 24.51 MiB | 2.84 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Updating files: 100% (87/87), done.


In [2]:
# 使用するjsonデータとdefine.xmlを新規ディレクトリにコピーする

import os
import shutil
import json

source_dir = "sdtm-adam-pilot-project/updated-pilot-submission-package/900172/m5/datasets/cdiscpilot01/tabulations/sdtm"
json_dir   = "json_files"
define_dir = "define_xml"

if not os.path.exists(json_dir):
    os.makedirs(json_dir)

if not os.path.exists(define_dir):
    os.makedirs(define_dir)

for root, _, files in os.walk(source_dir):
  for file in files:
    if file.endswith(".json"):
      source_path = os.path.join(root, file)
      target_path = os.path.join(json_dir, file)
      shutil.copy(source_path, target_path)
    if file.endswith("define.xml"):
      source_path = os.path.join(root, file)
      target_path = os.path.join(define_dir, file)
      shutil.copy(source_path, target_path)

In [3]:
# jsonファイルをリスト形式に結合したファイル（dataset_list.json）を作成

dataset_list = []
for filename in os.listdir(json_dir):
  if filename.endswith(".json"):
    with open(os.path.join(json_dir, filename), "r") as f:
      try:
        json_data = json.load(f)
        dataset_list.append(json_data)
      except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {filename}: {e}")

with open("dataset_list.json", "w") as f:
  json.dump(dataset_list, f)


## 症例フィルタリング関数の定義

In [4]:
def filter_data(data, target_usubjids):
    """
    複数のドメインデータを含むリストから、指定されたUSUBJIDのrowsのみを抽出して新しいJSONファイルに保存する。
    入力データがリストでない場合はエラーメッセージを出力する。
    データ構造は、"columns" 内の "name" が "USUBJID" の列を持つことを前提とする。

    Args:
        data (list): ドメインを結合させたのリスト。リストでない場合はエラーとなる。
        output_file (str): 出力するJSONファイル名。
        target_usubjids (list): 残したいUSUBJIDのリスト。
    """
    if not isinstance(data, list):
        print("エラー：入力データはJSONオブジェクトのリストである必要があります。")
        return

    filtered_data_list = []
    for item in data:
        usubjid_index = -1
        if 'columns' in item:
            for i, col in enumerate(item['columns']):
                if 'name' in col and col['name'] == 'USUBJID':
                    usubjid_index = i
                    break

        if usubjid_index == -1:
            print(f"警告：データセット '{item.get('fileOID', '不明')}' に 'name' が 'USUBJID' の列が見つかりません。スキップします。")
            filtered_data_list.append(item)
            continue

        if 'rows' in item:
            filtered_rows = [
                row for row in item['rows'] if len(row) > usubjid_index and row[usubjid_index] in target_usubjids
            ]
            new_data = item.copy()
            new_data['rows'] = filtered_rows
            new_data['records'] = len(filtered_rows)
            filtered_data_list.append(new_data)
        else:
            print(f"警告：データセット '{item.get('fileOID', '不明')}' に 'rows' が見つかりません。スキップします。")
            filtered_data_list.append(item)

    return filtered_data_list



# 実行テスト
# with open('dataset_list.json', 'r') as f:
#   data = json.load(f)
#
# target_ids = ['01-701-1211']
# output_filename = 'filtered_list.json'
#
# filtered_data_list = filter_data(data, target_ids)
#
# with open(output_filename, 'w') as f:
#   json.dump(filtered_data_list, f)
#
# print(f"処理完了：'{output_filename}' に USUBJID が {target_ids} のデータを出力しました。")

## データ書き換え関数の定義

In [5]:
def data_update(data, target_domain, target_usubjid, target_seq, target_variable, new_value):
    """
    指定されたUSUBJIDを持つレコードの指定された変数を書き換えます。
    {target_domain}SEQが存在する場合はそれもキーとして使用します。
    元のデータは変更せず、新しいデータ構造を返します。

    Args:
        data (list): データ全体のリスト。指定されたtarget_domainのデータセットを含むことを想定します。
        target_domain (str): 対象のドメイン名（例: "CM"）。
        target_usubjid (str): 書き換えたいレコードのUSUBJID。
        target_seq (int): 書き換えたいレコードの{target_domain}SEQの値（存在しない場合は無視されます）。
        target_variable (str): 書き換えたい変数の名前（例: "CMTRT"）。
        new_value (any): 新しい変数の値。

    Returns:
        list: 指定された変数が更新された新しいデータ全体のリスト。
              該当するレコードが見つからなかった場合、元のデータのコピーを返します。
    """
    updated_data = []
    seqname = target_domain + 'SEQ'

    for dataset in data:
        updated_dataset = dataset.copy()
        if updated_dataset.get("itemGroupOID") == target_domain:
            updated_rows = []
            found = False
            usubjid_index = -1
            seq_index = -1
            variable_index = -1
            has_seq = False

            for i, col in enumerate(updated_dataset["columns"]):
                if col["name"] == "USUBJID":
                    usubjid_index = i
                elif col["name"] == seqname:
                    seq_index = i
                    has_seq = True
                elif col["name"] == target_variable:
                    variable_index = i

            if usubjid_index != -1 and variable_index != -1:
                for row in dataset["rows"]:
                    updated_row = list(row)  # 行をコピーして変更
                    usubjid_match = updated_row[usubjid_index] == target_usubjid
                    seq_match = True
                    if has_seq and seq_index != -1:
                        seq_match = (len(updated_row) > seq_index and updated_row[seq_index] == target_seq)
                    elif has_seq:
                        print(f"警告: '{target_domain}' データセットに '{seqname}' 列が見つかりましたが、インデックスが無効です。USUBJIDのみをキーとして使用します。")

                    if usubjid_match and seq_match:
                        updated_row[variable_index] = new_value
                        if has_seq and seq_index != -1:
                            print(f"USUBJID '{target_usubjid}'、'{seqname}' '{target_seq}' の '{target_variable}' を '{new_value}' に更新しました。")
                        else:
                            print(f"USUBJID '{target_usubjid}' の '{target_variable}' を '{new_value}' に更新しました。")
                        found = True
                    updated_rows.append(updated_row)
                updated_dataset["rows"] = updated_rows
            elif updated_dataset.get("itemGroupOID") == target_domain:
                print(f"'{target_domain}' データセットに 'USUBJID' または '{target_variable}' 列が見つかりませんでした。")

            updated_data.append(updated_dataset)
            if not found and updated_dataset.get("itemGroupOID") == target_domain:
                if has_seq and seq_index != -1:
                    print(f"USUBJID '{target_usubjid}'、'{seqname}' '{target_seq}' に該当するレコードが見つかりませんでした。")
                else:
                    print(f"USUBJID '{target_usubjid}' に該当するレコードが見つかりませんでした。")
        else:
            updated_data.append(updated_dataset)

    if not any(d.get("itemGroupOID") == target_domain for d in data):
        print(f"{target_domain} データセットが見つかりませんでした。")

    return updated_data

# 書き換えテスト
# updated_data = data_update(filtered_data_list, "DM", "01-701-1211", 0, "AGE", 49)
# updated_data = data_update(updated_data, "CM", "01-701-1211", 3, "CMTRT", "New Drug 123456789")
# updated_data = data_update(updated_data, "CM", "01-701-1211", 0, "CMDOSE", 123)

## データ比較関数の定義

In [6]:
from typing import List, Dict, Any

def compare_data(old_data: List[Dict[str, Any]], new_data: List[Dict[str, Any]]) -> None:
    """
    2つのデータリストの更新差分を人間が読みやすい形式で出力します。

    Args:
        old_data: 旧データリスト。
        new_data: 新データリスト。
    """

    def create_row_dict(item_group: Dict[str, Any], row: List[Any]) -> Dict[str, Any]:
        """rowデータをキー付きの辞書に変換する"""
        row_dict = {}
        for i, column in enumerate(item_group['columns']):
            row_dict[column['name']] = row[i]
        return row_dict

    def get_key_values(item_group_oid: str, row_dict: Dict[str, Any]) -> Dict[str, Any]:
        """データのキーとなる値を抽出する"""
        key_values = {'USUBJID': row_dict.get('USUBJID')}
        seq_key = f"{item_group_oid}SEQ"
        if seq_key in row_dict:
            key_values[seq_key] = row_dict[seq_key]
        return key_values

    def format_key(key_values: Dict[str, Any]) -> str:
        """キー値を人間が読みやすい文字列に整形する"""
        parts = []
        for key, value in key_values.items():
            if value is not None:
                parts.append(f"{key} = {value}")
        return ", ".join(parts)

    old_data_by_group = {item['itemGroupOID']: item for item in old_data}
    new_data_by_group = {item['itemGroupOID']: item for item in new_data}

    all_group_oids = set(old_data_by_group.keys()) | set(new_data_by_group.keys())

    for group_oid in sorted(list(all_group_oids)):
        print(f"--- ItemGroupOID: {group_oid} ---")
        old_group = old_data_by_group.get(group_oid)
        new_group = new_data_by_group.get(group_oid)

        old_rows_by_key = {}
        if old_group and 'rows' in old_group:
            for row in old_group['rows']:
                row_dict = create_row_dict(old_group, row)
                if 'USUBJID' in row_dict and row_dict['USUBJID'] is not None:
                    key_values = get_key_values(group_oid, row_dict)
                    old_rows_by_key[format_key(key_values)] = row_dict

        new_rows_by_key = {}
        if new_group and 'rows' in new_group:
            for row in new_group['rows']:
                row_dict = create_row_dict(new_group, row)
                if 'USUBJID' in row_dict and row_dict['USUBJID'] is not None:
                    key_values = get_key_values(group_oid, row_dict)
                    new_rows_by_key[format_key(key_values)] = row_dict

        old_keys = set(old_rows_by_key.keys())
        new_keys = set(new_rows_by_key.keys())

        # 追加されたデータ
        added_keys = new_keys - old_keys
        for key in sorted(list(added_keys)):
            print(f"{key}:")
            print("  Added")
            for item_key, old_value in sorted(new_rows_by_key[key].items()):
                print(f"    {item_key}: {old_value}")
            print()

        # 削除されたデータ
        removed_keys = old_keys - new_keys
        for key in sorted(list(removed_keys)):
            print(f"{key}:")
            print("  Deleted")
            for item_key, old_value in sorted(old_rows_by_key[key].items()):
                print(f"    {item_key}: {old_value}")
            print()

        # 更新されたデータ
        common_keys = old_keys & new_keys
        for key in sorted(list(common_keys)):
            if old_rows_by_key[key] != new_rows_by_key[key]:
                print(f"{key}:")
                print("  Updated:")
                old_row = old_rows_by_key[key]
                new_row = new_rows_by_key[key]
                for item_key in sorted(list(set(old_row.keys()) | set(new_row.keys()))):
                    old_value = old_row.get(item_key)
                    new_value = new_row.get(item_key)
                    if old_value != new_value:
                        print(f"    {item_key}: {old_value!r} -> {new_value!r}")
                print()


# 比較テスト
# compare_data(filtered_data_list, updated_data)

In [7]:
import json

usubjids = set()
for dataset in dataset_list:
    if 'columns' in dataset:
        for i, col in enumerate(dataset['columns']):
            if 'name' in col and col['name'] == 'USUBJID':
                if 'rows' in dataset:
                    for row in dataset['rows']:
                        if len(row) > i:
                            usubjids.add(row[i])

print(list(usubjids))


['01-703-1042', '01-708-1178', '01-704-1233', '01-704-1127', '01-709-1312', '01-710-1053', '01-715-1085', '01-705-1393', '01-704-1120', '01-709-1020', '01-710-1137', '01-705-1431', '01-710-1278', '01-716-1151', '01-708-1158', '01-710-1376', '01-718-1172', '01-716-1331', '01-705-1059', '01-701-1287', '01-710-1249', '01-701-1429', '01-707-1276', '01-709-1238', '01-714-1375', '01-702-1082', '01-704-1332', '01-703-1119', '01-704-1114', '01-710-1408', '01-701-1345', '01-701-1153', '01-704-1266', '01-703-1379', '01-718-1066', '01-704-1241', '01-711-1433', '01-708-1406', '01-716-1177', '01-701-1145', '01-705-1377', '01-715-1321', '01-703-1403', '01-704-1074', '01-710-1154', '01-703-1295', '01-715-1405', '01-718-1150', '01-709-1301', '01-718-1254', '01-710-1183', '01-704-1445', '01-701-1028', '01-710-1270', '01-705-1303', '01-709-1168', '01-708-1032', '01-711-1284', '01-701-1181', '01-708-1316', '01-708-1352', '01-707-1430', '01-710-1315', '01-705-1421', '01-715-1134', '01-715-1225', '01-708-1

# データの書き換え

In [8]:
Target_data = [
["DM", "01-703-1096",   0, "AGE", 49],
["LB", "01-703-1042",   3, "LBORRES", "135"],
["LB", "01-703-1042",   4, "LBORRES", "145"],
["LB", "01-703-1086",  37, "LBORRES", "1"],
["LB", "01-703-1086",  72, "LBORRES", "1.2"],
["LB", "01-703-1086", 102, "LBORRES", "1.1"],
["LB", "01-703-1086", 132, "LBORRES", "1"],
["LB", "01-703-1086", 162, "LBORRES", "1.3"],
["LB", "01-703-1086", 197, "LBORRES", "0.9"],
["LB", "01-703-1086", 232, "LBORRES", "0.8"],
["LB", "01-703-1042",   3, "LBSTRESC", "135"],
["LB", "01-703-1042",   4, "LBSTRESC", "145"],
["LB", "01-703-1086",  37, "LBSTRESC", "1"],
["LB", "01-703-1086",  72, "LBSTRESC", "1.2"],
["LB", "01-703-1086", 102, "LBSTRESC", "1.1"],
["LB", "01-703-1086", 132, "LBSTRESC", "1"],
["LB", "01-703-1086", 162, "LBSTRESC", "1.3"],
["LB", "01-703-1086", 197, "LBSTRESC", "0.9"],
["LB", "01-703-1086", 232, "LBSTRESC", "0.8"],
["LB", "01-703-1042",   3, "LBSTRESN", 135],
["LB", "01-703-1042",   4, "LBSTRESN", 145],
["LB", "01-703-1086",  37, "LBSTRESN", 1],
["LB", "01-703-1086",  72, "LBSTRESN", 1.2],
["LB", "01-703-1086", 102, "LBSTRESN", 1.1],
["LB", "01-703-1086", 132, "LBSTRESN", 1],
["LB", "01-703-1086", 162, "LBSTRESN", 1.3],
["LB", "01-703-1086", 197, "LBSTRESN", 0.9],
["LB", "01-703-1086", 232, "LBSTRESN", 0.8],
["LB", "01-703-1042",   3, "LBNRIND", "HIGH"],
["LB", "01-703-1042",   4, "LBNRIND", "HIGH"],
["LB", "01-703-1086",  37, "LBNRIND", "LOW"],
["LB", "01-703-1086",  72, "LBNRIND", "LOW"],
["LB", "01-703-1086", 102, "LBNRIND", "LOW"],
["LB", "01-703-1086", 132, "LBNRIND", "LOW"],
["LB", "01-703-1086", 162, "LBNRIND", "LOW"],
["LB", "01-703-1086", 197, "LBNRIND", "LOW"],
["LB", "01-703-1086", 232, "LBNRIND", "LOW"],
["MH", "01-701-1097",   1, "MHTERM", "Loss of consciousness (Passed out)"],
["MH", "01-701-1097",   1, "MHSTDTC", "2023-01-01"],
["MH", "01-701-1111",   1, "MHTERM", "HEARING LOSS"],
["MH", "01-701-1180",   1, "MHTERM", "DEPRESSION (ANXIETY)"],
["MH", "01-702-1082",   1, "MHTERM", "Premenstrual pain"],
["MH", "01-703-1076",   1, "MHTERM", "Atrioventricular block (scheduled cardiac pacemaker insertion)"],
["MH", "01-703-1279",   1, "MHTERM", "schizophreniform disorders"],
["MH", "01-703-1299",   1, "MHTERM", "Cyclothymic disorder"],
["VS", "01-701-1047",  17, "VSORRES", "121"],
["VS", "01-701-1047",  18, "VSORRES", "124"],
["VS", "01-701-1047",  66, "VSORRES", "185"],
["VS", "01-701-1047",  67, "VSORRES", "183"],
["VS", "01-701-1383",  37, "VSORRES", "98"],
["VS", "01-701-1383", 122, "VSORRES", "160"],
["VS", "01-701-1387",   1, "VSORRES", "146"],
["VS", "01-701-1387",  32, "VSORRES", "72"],
["VS", "01-701-1047",  17, "VSSTRESC", "121"],
["VS", "01-701-1047",  18, "VSSTRESC", "124"],
["VS", "01-701-1047",  66, "VSSTRESC", "185"],
["VS", "01-701-1047",  67, "VSSTRESC", "183"],
["VS", "01-701-1383",  37, "VSSTRESC", "98"],
["VS", "01-701-1383", 122, "VSSTRESC", "160"],
["VS", "01-701-1387",   1, "VSSTRESC", "146"],
["VS", "01-701-1387",  32, "VSSTRESC", "72"],
["VS", "01-701-1047",  17, "VSSTRESN", 121],
["VS", "01-701-1047",  18, "VSSTRESN", 124],
["VS", "01-701-1047",  66, "VSSTRESN", 185],
["VS", "01-701-1047",  67, "VSSTRESN", 183],
["VS", "01-701-1383",  37, "VSSTRESN", 98],
["VS", "01-701-1383", 122, "VSSTRESN", 160],
["VS", "01-701-1387",   1, "VSSTRESN", 146],
["VS", "01-701-1387",  32, "VSSTRESN", 72],
["EX", "01-701-1148",   2, "EXDOSE", 82],
["EX", "01-701-1148",   3, "EXDOSE", 216],
["EX", "01-703-1258",   2, "EXDOSE", 27],
["CM", "01-701-1146",  29, "CMTRT", "PAROXETINE"],
["QS", "01-701-1023",1010, "QSORRES", "PRESENT"],
["QS", "01-701-1023",1012, "QSORRES", "PRESENT"],
["QS", "01-701-1111",5004, "QSORRES", "4"],
["QS", "01-701-1111",5019, "QSORRES", "4"],
["QS", "01-701-1111",5012, "QSORRES", "4"],
["QS", "01-701-1111",5027, "QSORRES", "4"],
["QS", "01-701-1118",6002, "QSORRES", "MARKED IMPROVEMENT"],
["QS", "01-701-1118",6003, "QSORRES", "MARKED WORSENING"],
["QS", "01-701-1181",4018, "QSORRES", "Y"],
["QS", "01-701-1181",4058, "QSORRES", "Y"],
["QS", "01-701-1181",4019, "QSORRES", "Y"],
["QS", "01-701-1181",4059, "QSORRES", "Y"],
["QS", "01-701-1181",4020, "QSORRES", "Y"],
["QS", "01-701-1023",1010, "QSSTRESC", "2"],
["QS", "01-701-1023",1012, "QSSTRESC", "2"],
["QS", "01-701-1111",5004, "QSSTRESC", "4"],
["QS", "01-701-1111",5019, "QSSTRESC", "4"],
["QS", "01-701-1111",5012, "QSSTRESC", "4"],
["QS", "01-701-1111",5027, "QSSTRESC", "4"],
["QS", "01-701-1118",6002, "QSSTRESC", "1"],
["QS", "01-701-1118",6003, "QSSTRESC", "7"],
["QS", "01-701-1181",4018, "QSSTRESC", "1"],
["QS", "01-701-1181",4058, "QSSTRESC", "1"],
["QS", "01-701-1181",4019, "QSSTRESC", "1"],
["QS", "01-701-1181",4059, "QSSTRESC", "1"],
["QS", "01-701-1181",4020, "QSSTRESC", "1"],
["QS", "01-701-1023",1010, "QSSTRESN", 2],
["QS", "01-701-1023",1012, "QSSTRESN", 2],
["QS", "01-701-1111",5004, "QSSTRESN", 4],
["QS", "01-701-1111",5019, "QSSTRESN", 4],
["QS", "01-701-1111",5012, "QSSTRESN", 4],
["QS", "01-701-1111",5027, "QSSTRESN", 4],
["QS", "01-701-1118",6002, "QSSTRESN", 1],
["QS", "01-701-1118",6003, "QSSTRESN", 7],
["QS", "01-701-1181",4018, "QSSTRESN", 1],
["QS", "01-701-1181",4058, "QSSTRESN", 1],
["QS", "01-701-1181",4019, "QSSTRESN", 1],
["QS", "01-701-1181",4059, "QSSTRESN", 1],
["QS", "01-701-1181",4020, "QSSTRESN", 1],
["QS", "01-701-1118",6001, "QSDTC", "2014-07-08"],
["QS", "01-701-1118",6001, "QSDY", 119],
["AE", "01-701-1015",   3, "AESER", "Y"],
["AE", "01-701-1015",   3, "AESHOSP", "Y"],
["AE", "01-701-1015",   3, "AESTDTC", "2014-01-11"],
["AE", "01-701-1015",   3, "AEENDTC", "2014-01-09"],
["AE", "01-701-1015",   3, "AESTDY", 10],
["AE", "01-701-1015",   3, "AEENDY", 8],
["AE", "01-701-1028",   1, "AETERM", "PARKINSON'S DISEASE"],
["AE", "01-701-1028",   1, "AESTDTC", "2013-07-01"],
["AE", "01-701-1028",   1, "AESTDY", -17],
["AE", "01-701-1034",   2, "AETERM", "MALIGNANT HYPERTENSION"],
["AE", "01-701-1047",   4, "AETERM", "HYPERTENSION"],
["AE", "01-701-1363",   1, "AESTDTC", "2013-06-15"],
["AE", "01-701-1363",   1, "AEENDTC", "2013-06-14"],
["AE", "01-701-1363",   1, "AESTDY", 17],
["AE", "01-701-1363",   1, "AEENDY", 16],
["AE", "01-701-1047",   3, "AEENDTC", "2013-03-05"],
["AE", "01-701-1047",   3, "AEENDY", 22],
["AE", "01-701-1383",  12, "AETERM", "BLOOD PRESSURE INCREASED"],
["AE", "01-701-1153",   2, "AEACN", "DRUG WITHDRAWN"],
["AE", "01-701-1180",   6, "AETERM", "SUDDEN DEATH"],
["AE", "01-703-1258",   2, "AESEV", "SEVERE"],
["AE", "01-703-1258",   2, "AESTDTC", "2012-08-01"],
["AE", "01-703-1258",   2, "AEENDTC", "2012-10-01"],
["AE", "01-703-1258",   2, "AESTDY", 13],
["AE", "01-703-1258",   2, "AEENDY", 74],
["AE", "01-703-1258",   5, "AESEV", "MODERATE"],
["AE", "01-703-1258",   5, "AESER", "Y"],
["AE", "01-703-1258",   5, "AEOUT", "RECOVERED/RESOLVED"],
["AE", "01-703-1258",   5, "AESLIFE", "Y"],
["AE", "01-703-1258",   5, "AESTDTC", "2012-10-02"],
["AE", "01-703-1258",   5, "AEENDTC", "2012-12-31"],
["AE", "01-703-1258",   2, "AESTDY", 75],
["AE", "01-703-1258",   2, "AEENDY", 165],
["AE", "01-703-1335",   1, "AETERM", "MULTIPLE SCLEROSIS RELAPSE"],
["AE", "01-703-1335",   1, "AESTDTC", "2014-04-01"],
["AE", "01-703-1335",   1, "AEENDTC", "2014-05-01"],
["AE", "01-703-1335",   1, "AESTDY", 15],
["AE", "01-703-1335",   1, "AEENDY", 46],
["AE", "01-703-1403",   2, "AETERM", "MYASTHENIA GRAVIS AGGRAVATED"],
["AE", "01-704-1008",   1, "AETERM", "TREMOR IN HANDS, LEGS"],
["AE", "01-704-1008",   1, "AEREL", "NONE"],
["AE", "01-704-1008",   1, "AESTDTC", "2012-06-01"],
["AE", "01-704-1008",   1, "AESTDY", -225],
["AE", "01-704-1008",   3, "AETERM", "MUSCLE STIFFNESS"],
["AE", "01-704-1008",   3, "AESTDTC", "2012-06-01"],
["AE", "01-704-1008",   3, "AESTDY", -225],
["AE", "01-704-1008",   2, "AETERM", "SLOWNESS of MOVEMENT"],
["AE", "01-704-1008",   2, "AESTDTC", "2012-06-01"],
["AE", "01-704-1008",   2, "AESTDY", -225],
["AE", "01-704-1009",   6, "AETERM", "CHRONIC KIDNEY DISEASE"],
["AE", "01-704-1009",   6, "AESER", "Y"],
["AE", "01-704-1009",   6, "AESLIFE", "Y"],
["AE", "01-704-1010",   1, "AETERM", "DIABETES MELLITUS"],
["AE", "01-704-1010",   1, "AESER", "Y"],
["AE", "01-704-1010",   1, "AESLIFE", "Y"],
["AE", "01-704-1017",   4, "AETERM", "LATE EFFECTS OF CEREBRAL INFRACTION"],
["AE", "01-704-1017",   4, "AESEV", "SEVERE",],
["AE", "01-704-1017",   4, "AESTDTC", "2013-10-19"],
["AE", "01-704-1017",   4, "AEENDTC", "2013-11-18"],
["AE", "01-704-1017",   4, "AESTDY", 14],
["AE", "01-704-1017",   4, "AEENDY", 44],
["AE", "01-704-1017",   3, "AETERM", "BRAIN DEATH"],
["AE", "01-704-1017",   3, "AESEV", "SEVERE",],
["AE", "01-704-1017",   3, "AESTDTC", "2013-11-18"],
["AE", "01-704-1017",   3, "AEENDTC", "2013-11-18"],
["AE", "01-704-1017",   3, "AESTDY", 44],
["AE", "01-704-1017",   3, "AEENDY", 44],
["AE", "01-704-1017",   1, "AEOUT", "RECOVERED/RESOLVED"],
["AE", "01-704-1017",   1, "AESTDTC", "2013-10-19"],
["AE", "01-704-1017",   1, "AEENDTC", "2013-11-19"],
["AE", "01-704-1017",   1, "AESTDY", 14],
["AE", "01-704-1017",   1, "AEENDY", 45],
["AE", "01-704-1017",   1, "AEACN", "DRUG WITHDRAWN"]
]

dataset_list_updated = dataset_list

for l in Target_data:
  #print(l)
  dataset_list_updated = data_update(dataset_list_updated, l[0], l[1], l[2], l[3], l[4])

USUBJID '01-703-1096' の 'AGE' を '49' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '3' の 'LBORRES' を '135' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '4' の 'LBORRES' を '145' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '37' の 'LBORRES' を '1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '72' の 'LBORRES' を '1.2' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '102' の 'LBORRES' を '1.1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '132' の 'LBORRES' を '1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '162' の 'LBORRES' を '1.3' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '197' の 'LBORRES' を '0.9' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '232' の 'LBORRES' を '0.8' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '3' の 'LBSTRESC' を '135' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '4' の 'LBSTRESC' を '145' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '37' の 'LBSTRESC' を '1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '72' の 'LBSTRESC' を '1.2' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '102' の 'LBSTRESC' を '1.1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '132' の 'LBSTRESC' を '1' に更

In [9]:
# 更新データ確認
compare_data(dataset_list, dataset_list_updated)

with open("dataset_list_updated.json", "w") as f:
  json.dump(dataset_list_updated, f)

--- ItemGroupOID: AE ---
USUBJID = 01-701-1015, AESEQ = 3:
  Updated:
    AEENDTC: '2014-01-11' -> '2014-01-09'
    AEENDY: 10 -> 8
    AESER: 'N' -> 'Y'
    AESHOSP: 'N' -> 'Y'
    AESTDTC: '2014-01-09' -> '2014-01-11'
    AESTDY: 8 -> 10

USUBJID = 01-701-1028, AESEQ = 1:
  Updated:
    AESTDTC: '2013-07-21' -> '2013-07-01'
    AESTDY: 3 -> -17
    AETERM: 'APPLICATION SITE ERYTHEMA' -> "PARKINSON'S DISEASE"

USUBJID = 01-701-1034, AESEQ = 2:
  Updated:
    AETERM: 'FATIGUE' -> 'MALIGNANT HYPERTENSION'

USUBJID = 01-701-1047, AESEQ = 3:
  Updated:
    AEENDTC: '' -> '2013-03-05'
    AEENDY: None -> 22

USUBJID = 01-701-1047, AESEQ = 4:
  Updated:
    AETERM: 'BUNDLE BRANCH BLOCK LEFT' -> 'HYPERTENSION'

USUBJID = 01-701-1153, AESEQ = 2:
  Updated:
    AEACN: '' -> 'DRUG WITHDRAWN'

USUBJID = 01-701-1180, AESEQ = 6:
  Updated:
    AETERM: 'MICTURITION URGENCY' -> 'SUDDEN DEATH'

USUBJID = 01-701-1363, AESEQ = 1:
  Updated:
    AEENDTC: '2013-06-15' -> '2013-06-14'
    AEENDY: 17 -> 16

# LLMへの送信

In [10]:
!pip install sseclient-py
import requests
import sseclient
from IPython.display import display, Markdown

from google.colab import userdata
api_key = userdata.get('Dify_DatasetJSON')
user_id = 'JPMA_Sample'

## 関数定義

In [11]:
import json
import time
import requests
import sseclient

# 定数
DIFY_API_URL = 'https://api.dify.ai/v1/workflows/run'
CONTENT_TYPE_JSON = 'application/json'

def call_dify_api(api_key: str, payload: dict, stream: bool = False) -> requests.Response:
    """Dify APIを呼び出す共通関数"""
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': CONTENT_TYPE_JSON
    }
    try:
        response = requests.post(DIFY_API_URL, headers=headers, json=payload, stream=stream)
        response.raise_for_status()  # HTTPエラーが発生した場合に例外を発生させる
        return response
    except requests.exceptions.RequestException as e:
        print(f"API呼び出しエラー: {e}")
        raise

def run_dify_workflow(api_key: str, workflow_inputs: dict, user_id: str, streaming: bool = False) -> dict | sseclient.Event:
    """Difyワークフローを実行する

    Args:
        api_key: Dify APIキー
        workflow_inputs: ワークフローへの入力
        user_id: ユーザーID
        streaming: ストリーミングモードで実行するかどうか (Falseの場合はブロッキングモード)

    Returns:
        ストリーミングモードの場合はsseclient.Eventのイテレータ、
        ブロッキングモードの場合はAPIのレスポンスのJSONを辞書型で返す
    """
    payload = {
        'inputs': workflow_inputs,
        'response_mode': 'streaming' if streaming else 'blocking',
        'user': user_id
    }
    response = call_dify_api(api_key, payload, stream=streaming)
    if streaming:
        client = sseclient.SSEClient(response)
        return client.events()
    else:
        return response.json()

def safe_print_event_data(event: sseclient.Event):
    """
    与えられたSSEイベントデータから、存在する場合に特定の値を出力します。
    キーが存在しない場合は何も出力しません。Statusが存在する場合にのみTitleと結合させて表示します。

    Args:
        event: イベントデータを含むSSEイベントオブジェクト。event.data属性がJSON文字列であることを想定。
    """
    try:
        data = json.loads(event.data)

        if 'event' in data:
            print(f"Event: {data['event']}")

        if 'data' in data:
            title = data['data'].get('title')
            status = data['data'].get('status')
            if title is not None and status is not None:
                print(f"Node: {title} ({status})")
            elif title is not None:
                print(f"Node: {title}") # Statusが存在しない場合はTitleのみ表示

            if 'error' in data['data']:
                print(f"Error: {data['data']['error']}")
            if 'elapsed_time' in data['data']:
                print(f"Elapsed time: {data['data'].get('elapsed_time')}")
            if 'total_tokens' in data['data']:
                print(f"Total tokens: {data['data'].get('total_tokens')}")

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON event data: {e}")
    except AttributeError as e:
        print(f"Error accessing event data attribute: {e}")

def run_workflow_with_retry(api_key: str, workflow_inputs: dict, user_id: str, max_retries: int = 3, retry_delay: int = 30):
    """ワークフローを実行し、エラー発生時にリトライを行う (ストリーミングモード専用)"""
    for retry in range(max_retries + 1):
        print(f"--- 試行回数: {retry + 1} ---")
        success = True
        try:
            for event in run_dify_workflow(api_key, workflow_inputs, user_id, streaming=True):
                safe_print_event_data(event)
                try:
                    event_data = json.loads(event.data)
                    if event_data.get('data', {}).get('error') is not None:
                        print(f"エラーが検出されました: {event_data['data']['error']}")
                        success = False
                        break
                except json.JSONDecodeError:
                    print("JSONデコードエラーが発生しました。")
                    success = False
                    break
                print('------')

            if success:
                print("ワークフローが正常に完了しました。")
                return json.loads(event.data)
            elif retry < max_retries:
                print(f"エラーが発生したため、{retry_delay}秒後に再試行します...")
                time.sleep(retry_delay)
            else:
                print("最大再試行回数に達しました。ワークフローは失敗しました。")
                return False

        except requests.exceptions.RequestException as e:
            print(f"APIリクエスト中にエラーが発生しました: {e}")
            success = False
            if retry < max_retries:
                print(f"{retry_delay}秒後に再試行します...")
                time.sleep(retry_delay)
            else:
                print("最大再試行回数に達しました。ワークフローは失敗しました。")
                return False

## プロンプトの作成

In [12]:
with open('define_xml/define.xml', 'r') as f:
  define_xml = f.read()


SysPrompt = '''
あなたは、臨床試験データのレビューを支援するAIアシスタントです。以下の前提知識を理解した上で、ユーザーからの指示（ユーザープロンプト）に従って、臨床試験データのレビューを支援してください。各タスクでは、ユーザープロンプトで指定された役割になりきって回答してください。

**前提知識:**

*   臨床試験においては患者の安全性が最優先され、有害事象の評価は特に重要です。
*   SDTM (Study Data Tabulation Model) は、CDISCによって策定された臨床試験データの標準モデルです。
*   Define.xmlはSDTMデータの構造を記述したメタデータファイルであり、参考情報として使用します。JSONデータ自体の内容、医学的妥当性、プロトコルとの整合性を優先してレビューしてください。
*   SDTMデータは、DM、AE、VS、LBなど、複数のドメイン（データセット）に分かれています。
*   報告されるJSONデータには、データ入力時の間違いが含まれる可能性があります。
*   提供された情報のみに基づいて回答を作成してください。想像やハルシネーションに基づいた回答は作成してはいけません。

**出力形式:**

*   すべての出力はMarkdown形式で作成してください。
*   見出し記号(#)は使用しないでください。

**その他:**

*   JSONデータまたはDefine.xmlの形式が不正な場合は、その旨をエラーメッセージとして出力してください。
'''




UserInput_Task1 = '''
あなたは臨床試験の専門医です。以下の指示に従い、提供される情報（プロトコル、JSONデータ、Define.xml）を基に、臨床試験データのレビューとクエリ作成（必要な場合）を行ってください。

**1. 症例サマリーの作成:**

*   **参照情報:** JSONデータ、Define.xml
*   **タスク:**
    *   JSONデータとDefine.xmlを参照し、有害事象、検査値、バイタルサインなどの推移を時系列でまとめた症例サマリーを作成してください。
    *   特に、**異常所見**を中心に簡潔な文章で記載してください。正常範囲内の変動は省略して構いません。
    *   各イベントの日時は、Define.xmlに定義された日付変数などを参考に、正確に特定してください。
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

    *   **患者ID:**
        *   YYYY年MM月DD日 (Day XX): [有害事象、検査値、バイタルサインなどのイベントを、異常所見を中心に簡潔な文章で記載]

---

**2. クエリの作成 (必要な場合のみ):**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   以下のJSONデータのレビュー観点に基づき、JSONデータを改めて点検してください。
    *   医療機関への問い合わせが必要な事項（疑義、不明点、確認事項など）が発生した場合、その内容をまとめたクエリを作成してください。
    *   クエリは、報告されたデータと、Define.xml、プロトコルの記述に基づいて作成してください。提供された情報から逸脱する内容や、想像、ハルシネーションに基づくクエリは作成してはいけません。
    *   クエリは、臨床試験の評価項目に対する影響度を考慮し、重要度の高いものから優先的に作成してください。
    *  **疑義事項がない場合は、クエリを作成する必要はありません。**「疑義事項なし」と回答してください。

*   **JSONデータのレビュー観点 (これらに限定されない):**
    *   **安全性:** 有害事象(AEドメイン)の報告内容は、医学的に妥当であるか？
    *   **医学的妥当性:** 検査値(LBドメイン)の変動、バイタルサイン(VSドメイン)の変動、併用薬(CMドメイン)との相互作用など、時間経過とともに医学的に問題となる点は見られるか？
    *   **有効性:** 特定された主要評価項目および副次評価項目について、その時間的変化は期待される効果と一致しているか？
    *   **その他:** 患者背景(DMドメイン)、既往歴(MHドメイン)、有害事象(AEドメイン)、治療歴(EXドメイン, CMドメイン)などを総合的に考慮し、時間経過を加味して安全性に懸念を生じる事項があれば記載してください。
    *   **プロトコル逸脱 (疑い):** 選択/除外基準、投与量、併用禁止薬、評価スケジュール、有害事象報告などについて、プロトコルからの逸脱の疑いがないか確認してください。（関連ドメイン: DM, MH, EX, CM, LB, VS, AEなど）

*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。クエリがない場合は、「クエリなし」と出力してください。

    *   **患者ID:**
        *   **クエリNo.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor/None）
            *   **変数名と値:**
            *   **医療機関への問い合わせ文面:**
            *   **判断理由:**
'''



UserInput_Task2 = '''
あなたはクリニカルデータマネージャーです。以下の指示に従い、提供される情報（JSONデータ、Define.xml、プロトコル）を基に、データ整合性レビューとクエリ作成（必要な場合）を行ってください。

**1. データ整合性レビュー:**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   JSONデータ、Define.xml、プロトコルを参照し、データの不整合が疑われる問題点を検出してください。
    *   **特に、以下の点に焦点を当ててレビューしてください。**
        *   **クロスドメイン整合性:** 異なるSDTMドメイン間で、データに矛盾がないか、ドメイン間の関連性が正しく表現されているか。
            *   **具体的な確認例 (これらに限定されない):**
                *   DM.SEXとAEにおける妊娠関連の有害事象
                *   AEの有害事象発現日や治験薬との関連性と、EXの治験薬の投与期間
                *   LBの検査値異常とAEの関連有害事象
                *   VSのバイタルサイン異常とAEの関連有害事象
                *   CM.CMTRTとAE/MHで報告されている疾患・既往歴との矛盾
        *   **単一ドメイン内の整合性:** Define.xmlの定義に照らして、矛盾なく解釈できるデータになっているか、プロトコルに照らしてデータの関連性が正しく表現されているか。
        *   **異常値:** Define.xmlで定義された範囲外、または医学的にありえない値がないか。
        *   **欠損値:** 欠損値の有無と理由（推測できる場合）。多い場合は原因を推測。
        *   **プロトコル逸脱 (データ品質の観点から):** データ入力/収集で、プロトコルからの逸脱（例：必須項目の未入力、不適切な時期のデータ収集）がないか。
    *   Define.xmlとデータの間に不整合がある場合は、「Define.xmlの修正候補」として報告してください。
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

    *   **全体的なデータ品質の評価:**
        *   総合評価: [例：良好、一部問題あり、要修正]
        *   データクリーニング/再調査が必要な項目: [該当項目を列挙]

    *   **問題点:**（問題がある場合）
        *   **問題No.:**
            *   **変数名と値:**
            *   **矛盾の内容:** [具体的な矛盾の内容を記述]
        *   **問題点の原因（推測）:**
        *   **対応策（提案）:**

**2. クエリの作成 (必要な場合のみ):**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   データ整合性レビューの結果、医療機関への問い合わせが必要な事項（疑義、不明点、確認事項など）が発生した場合、その内容をまとめたクエリを作成してください。
    *   クエリは、報告されたデータと、Define.xml、プロトコルの記述に基づいて作成してください。提供された情報から逸脱する内容や、想像、ハルシネーションに基づくクエリは作成してはいけません。
    *   クエリは、臨床試験の評価項目に対する影響度を考慮し、重要度の高いものから優先的に作成してください。
    *   **疑義事項がない場合は、クエリを作成する必要はありません。**
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。クエリがない場合は、「クエリなし」と出力してください。

    *   **患者ID:**
        *   **クエリNo.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor/None）
            *   **変数名と値:**
            *   **医療機関への問い合わせ文面:**
            *   **判断理由:**
'''


UserInput_Task3 = '''
あなたは、臨床試験の専門医、データマネージャー、CRAの視点を持つ、プロトコル遵守状況の確認者です。以下の指示に従い、提供される情報（JSONデータ、Define.xml、プロトコル）を基に、プロトコル逸脱の検出とクエリ作成（必要な場合）を行ってください。

**1. プロトコル逸脱の検出:**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   JSONデータ、Define.xml、プロトコルを参照し、プロトコルからの逸脱を検出してください。
    *   Define.xmlは参考情報として活用し、データとプロトコルの内容を比較して逸脱を判断してください。
    *   **検出対象とすべき主要なプロトコル逸脱の例 (これらに限定されない):**
        *   **選択/除外基準違反:** (関連SDTMドメイン: DM, MH など)
        *   **投与量違反:** (関連SDTMドメイン: EX)
        *   **併用禁止薬の使用:** (関連SDTMドメイン: CM)
        *   **評価スケジュール違反:** (関連SDTMドメイン: LB, VS, その他)
        *   **有害事象報告違反**: (関連SDTMドメイン: AE)
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

    *   **患者ID:**
        *   **逸脱No.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor）
            *   **変数名と値:**
            *   **逸脱内容:** [具体的な逸脱内容を簡潔に記述。例：被験者XXXは、プロトコルで規定された投与量を超える量の治験薬を投与された]
            *   **プロトコル該当箇所:** [プロトコルの該当するセクション、ページ番号などを記載]
            *   **判断理由:**

**2. クエリの作成 (必要な場合のみ):**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   プロトコル逸脱を判定するために、医療機関への問い合わせが必要な事項（疑義、不明点、確認事項など）が発生した場合、その内容をまとめたクエリを作成してください。
    *   クエリは、報告されたデータと、Define.xml、プロトコルの記述に基づいて作成してください。提供された情報から逸脱する内容や、想像、ハルシネーションに基づくクエリは作成してはいけません。
    *   クエリは、プロトコル逸脱が臨床試験の評価項目に与える影響度を考慮し、重要度の高いものから優先的に作成してください。
    *   **プロトコル逸脱に関する疑義事項がない場合は、クエリを作成する必要はありません。**
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。クエリがない場合は、「クエリなし」と出力してください。

    *   **患者ID:**
        *   **クエリNo.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor/None）
            *   **変数名と値:**
            *   **医療機関への問い合わせ文面:**
            *   **判断理由:**
'''


UserInput_end1 = '''\n---\n\n**データ:**\n\n*   臨床試験データ（JSON形式、SDTM準拠）:\n\n```json\n'''
UserInput_end2 = '''\n```\n\n*   データ定義ファイル（Define.xml）:\n\n```xml\n''' + define_xml + '''```\n'''

In [13]:
def create_workflow_input(ModelName, SysPrompt, UserInput_Task, datasetjson, UserInput_end1, UserInput_end2):
    return {
        'ModelName': ModelName,
        'SysPrompt': SysPrompt,
        'UserInput': UserInput_Task + UserInput_end1 + datasetjson + UserInput_end2,
        'AttachedFile': {"type": "document", "transfer_method": "local_file", "upload_file_id": "6b06d4f8-d47a-441f-bf67-d8700f76f556"}
    }

## 実行

In [14]:
# ModelNameの設定
#ModelName = 'gemini-2.0-flash'
#ModelName = 'gemini-2.0-flash-exp'
#ModelName = 'gemini-2.0-pro-exp-02-05'
#ModelName = 'gemini-2.0-flash-thinking-exp-01-21'
ModelName = 'gemini-2.0-flash-thinking-exp'


# データ更新症例の抽出
updated_subjects = []
for l in Target_data:
  updated_subjects.append(l[1])

updated_subjects = list(set(updated_subjects))
print(updated_subjects)


['01-703-1042', '01-701-1146', '01-703-1086', '01-703-1096', '01-703-1258', '01-701-1180', '01-701-1015', '01-704-1009', '01-704-1017', '01-701-1023', '01-701-1383', '01-702-1082', '01-701-1118', '01-701-1047', '01-701-1153', '01-701-1111', '01-703-1335', '01-704-1010', '01-703-1403', '01-701-1028', '01-701-1148', '01-701-1181', '01-701-1387', '01-704-1008', '01-703-1299', '01-701-1034', '01-703-1279', '01-703-1076', '01-701-1363', '01-701-1097']


In [15]:
import pandas as pd

# リトライ付きでワークフローを実行
results_list = []

for subj in updated_subjects[0:2]:
    datasetjson = filter_data(dataset_list_updated, subj)
    print(f"処理完了：'datasetjson' に USUBJID が {subj} のデータを出力しました。")

    row_data = {'Subject': subj}  # 各行のデータを格納する辞書

    # Task 1 の処理
    workflow_inputs_Task1 = create_workflow_input(ModelName, SysPrompt, UserInput_Task1, UserInput_end1, json.dumps(datasetjson), UserInput_end2)
    result_Task1 = run_workflow_with_retry(api_key, workflow_inputs_Task1, user_id)
    output_Task1 = result_Task1['data']['outputs']['text']
    display(Markdown(output_Task1))
    row_data['Task1'] = output_Task1

    # Task 2 の処理
    workflow_inputs_Task2 = create_workflow_input(ModelName, SysPrompt, UserInput_Task2, UserInput_end1, json.dumps(datasetjson), UserInput_end2)
    result_Task2 = run_workflow_with_retry(api_key, workflow_inputs_Task2, user_id)
    output_Task2 = result_Task2['data']['outputs']['text']
    display(Markdown(output_Task2))
    row_data['Task2'] = output_Task2

    # Task 3 の処理
    workflow_inputs_Task3 = create_workflow_input(ModelName, SysPrompt, UserInput_Task3, UserInput_end1, json.dumps(datasetjson), UserInput_end2)
    result_Task3 = run_workflow_with_retry(api_key, workflow_inputs_Task3, user_id)
    output_Task3 = result_Task3['data']['outputs']['text']
    display(Markdown(output_Task3))
    row_data['Task3'] = output_Task3

    results_list.append(row_data)

# DataFrameを作成
df_results = pd.DataFrame(results_list)

# DataFrameを表示
display(df_results)

警告：データセット 'CDISCPILOT01.ta' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.te' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.ti' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.ts' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.tv' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
処理完了：'datasetjson' に USUBJID が 01-703-1042 のデータを出力しました。
--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.052911
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.349406
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.266742
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started

患者ID: 01-703-1042
* 2012年12月27日 (Day -65): 
    * ALT (アラニンアミノトランスフェラーゼ) が基準値上限を超えています(135 U/L [基準値上限: 43 U/L])。
    * AST (アスパラギン酸アミノトランスフェラーゼ) が基準値上限を超えています(145 U/L [基準値上限: 36 U/L])。
    * Anisocytes (異形赤血球) が認められました (1 NO UNITS [基準値: 正常範囲内])。
* 2013年02月21日 (Day -9): ナトリウム (Sodium) が基準値下限を下回っています(133 mEq/L [基準値下限: 135 mEq/L])。
* 2013年03月04日 (Day 3): 下痢（軽度）を発症しました。（治験薬との関連性：POSSIBLE、転帰：RECOVERED/RESOLVED）
* 2013年03月05日 (Day 4): 不眠症（軽度）を発症しました。（治験薬との関連性：REMOTE、転帰：RECOVERED/RESOLVED）、下痢 改善
* 2013年03月28日 (Day 27): Ery. Mean Corpuscular Volume (平均赤血球容積, MCV) が基準値上限を超えています(101 fL [基準値上限: 100 fL])。
* 2013年08月31日 (Day 183): AST (アスパラギン酸アミノトランスフェラーゼ) が基準値上限を超えています(38 U/L [基準値上限: 36 U/L])。

---
患者ID: 01-703-1042
* クエリNo.: 1
    * 臨床試験結果への影響度合い: Major
    * 変数名と値: LB.LBTESTCD=ALT, LBORRES=135 U/L (Day -65), LB.LBTESTCD=AST, LBORRES=145 U/L (Day -65)
    * 医療機関への問い合わせ文面: スクリーニング時のALT値およびAST値高値の原因について、医療機関に確認してください。治験薬投与の可否に影響を与える可能性があります。
    * 判断理由: スクリーニング検査時のALT値およびAST値が基準範囲上限を大きく超えています。肝機能障害の可能性や、治験薬投与による肝機能への影響を懸念する必要があります。治験薬投与開始前に、ALT/AST高値の原因特定と、治験薬投与の可否について専門家の判断が必要と考えられます。

* クエリNo.: 2
    * 臨床試験結果への影響度合い: Minor
    * 変数名と値: CMENDTC (CMドメイン)
    * 医療機関への問い合わせ文面: 併用薬NORVASC (AMLODIPINE), VITAMIN C, VITAMIN E の投与終了日 (CMENDTC) が空欄となっています。投与は継続中でしょうか？投与終了日を教えてください。
    * 判断理由: CMENDTCが空欄である場合、併用薬の投与状況が不明確であり、データの正確性に疑義が生じます。併用薬の投与状況は、治験薬の効果や有害事象の評価に影響を与える可能性があるため、医療機関に確認し、データの修正が必要です。

* クエリNo.: 3
    * 臨床試験結果への影響度合い: Minor
    * 変数名と値: LB.LBTESTCD=SODIUM, LBORRES=133 mEq/L (Day -9)
    * 医療機関への問い合わせ文面: 治験薬投与前検査 (Day -9) において、血清ナトリウム値が基準値下限を下回っています (133 mEq/L)。臨床的な意義と、治験薬投与に影響がないか確認してください。
    * 判断理由: 治験薬投与開始前の検査で低ナトリウム血症が確認されました。一般的に軽度な低ナトリウム血症は臨床的に問題とならないことが多いですが、患者の基礎疾患や併用薬によっては注意が必要な場合があります。念のため、治験責任医師に確認し、治験薬投与に影響がないか確認することが望ましいと考えられます。

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.137196
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 2.52073
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.173706
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T:0.3) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T:0.5) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T:0.7) gemini-2.0-flash-thinking-exp 
------
Event: node_fi

全体的なデータ品質の評価:
* 総合評価: 一部問題あり
* データクリーニング/再調査が必要な項目: CM.CMSTDTC, CM.CMDTC, CM.CMENDTC, LB.LBORRES (ALT, AST, ANISO, CHOL, SODIUM), LB.LBORNRLO, LB.LBORNRHI, LB.LBSTNRLO, LB.LBSTNRHI, LB.LBBLFL, QS.QSBLFL, DM.RFICDTC, DM.DTHDTC, DM.DTHFL, LBのUNSCHEDULED VISITレコード

問題点:
* 問題No.: 1
    * 変数名と値: LB.LBTESTCD=ALT, LB.LBORRES=135, LB.LBSTRESC=135, LB.LBNRIND=HIGH (SCREENING 1)
    * 矛盾の内容: スクリーニング検査において、ALT値が基準範囲上限を超えている。肝機能障害や薬剤性肝障害の可能性、またはデータ入力誤りの懸念があります。
    * 問題点の原因（推測）: 患者の肝機能異常、データ入力エラー
    * 対応策（提案）: 医療機関にALT高値の原因、医学的妥当性を確認。MH/CMドメインでALT上昇に関連する既往歴や併用薬の確認。肝機能関連検査値の確認。再検査の要否検討。Define-XML定義とデータ整合性確認。

* 問題No.: 2
    * 変数名と値: LB.LBTESTCD=AST, LB.LBORRES=145, LB.LBSTRESC=145, LB.LBNRIND=HIGH (SCREENING 1)
    * 矛盾の内容: スクリーニング検査において、AST値が基準範囲上限を超えている。肝機能障害や薬剤性肝障害の可能性、またはデータ入力誤りの懸念があります。
    * 問題点の原因（推測）: 患者の肝機能異常、データ入力エラー
    * 対応策（提案）: 医療機関にAST高値の原因、医学的妥当性を確認。MH/CMドメインでAST上昇に関連する既往歴や併用薬の確認。肝機能関連検査値の確認。再検査の要否検討。Define-XML定義とデータ整合性確認。

* 問題No.: 3
    * 変数名と値: LB.LBTESTCD=ANISO, LB.LBORRES=1, LB.LBSTRESC=1, LB.LBNRIND=ABNORMAL (WEEK 4)
    * 矛盾の内容: WEEK 4の検査において、ANISO（Anisocytes：赤血球不同症）がABNORMALとなっている。貧血や溶血性疾患の可能性、またはデータ入力誤りの懸念があります。
    * 問題点の原因（推測）: 患者の血液データ異常、データ入力エラー
    * 対応策（提案）: 医療機関にANISO異常値の原因、医学的妥当性を確認。Hematology関連の他の検査値も確認。MH/CMドメインでANISO異常に関連する既往歴や併用薬の確認。再検査の要否検討。Define-XML定義とデータ整合性確認。

* 問題No.: 4
    * 変数名と値: LB.LBTESTCD=CHOL, LBORRES=208 (UNSCHEDULED 1.1), 202 (WEEK 2), 199 (WEEK 4), ..., 201 (WEEK 26), LBORNRLO=149, LBORNRHI=286, LBNRIND=NORMAL
    * 矛盾の内容: コレステロール値（CHOL）が基準範囲内だが、測定時期を通して変動が少ない。測定値の信頼性、または患者の病態変化が少ない可能性について確認が必要です。
    * 問題点の原因（推測）: データ入力エラー、測定機器の不具合、または患者のコレステロール値が安定している
    * 対応策（提案）: 医療機関にコレステロール値の変動が小さい理由、測定方法の妥当性を確認。データ入力値の再確認。

* 問題No.: 5
    * 変数名と値: LB.LBTESTCD=SODIUM, LB.LBORRES=133, LB.LBSTRESC=133, LB.LBNRIND=LOW (UNSCHEDULED 1.1)
    * 矛盾の内容: UNSCHEDULED VISIT 1.1の検査において、SODIUM値が基準範囲下限を下回っている。電解質異常、脱水、SIADH（抗利尿ホルモン不適合分泌症候群）などの可能性、またはデータ入力誤りの懸念があります。
    * 問題点の原因（推測）: 患者の電解質異常、データ入力エラー
    * 対応策（提案）: 医療機関にSODIUM低値の原因、医学的妥当性を確認。電解質関連の他の検査値も確認。AEドメインで電解質異常に関連する有害事象の有無を確認。CMドメインで利尿剤等の投与状況を確認。再検査の要否検討。Define-XML定義とデータ整合性確認。

* 問題No.: 6
    * 変数名と値: CM.CMSTDTC = 2013-01-27, CM.CMDTC = 2013-02-23 (複数レコードで発生)
    * 矛盾の内容: 併用薬開始日 (CMSTDTC) がデータ収集日 (CMDTC) より前の日付になっている。時間軸の矛盾であり、データ収集または入力時のエラーが疑われます。
    * 問題点の原因（推測）: データ入力時の誤り、データ収集日の誤り
    * 対応策（提案）: 医療機関にCMドメインの該当レコードのCMSTDTCとCMDTCの日付の正当性を確認。

* 問題No.: 7
    * 変数名と値: CM.CMENDTC (null)
    * 矛盾の内容: 併用薬（NORVASC, VITAMIN C, VITAMIN E）のCMENDTC（Medication End Date/Time）が欠損しているレコードが複数存在する。併用薬の使用状況が不明確になるため、データの正確性に疑義が生じます。
    * 問題点の原因（推測）: 併用薬が継続中、データ入力時の未入力
    * 対応策（提案）: 医療機関にCMENDTCが欠損している理由を確認し、必要であればデータ修正を依頼。Define.xmlでCMENDTCの必須性を確認。

* 問題No.: 8
    * 変数名と値: DM.RFICDTC, DM.DTHDTC, DM.DTHFL (欠損値)
    * 矛盾の内容: DMドメインのRFICDTC（IC同意取得日）, DTHDTC（死亡日）, DTHFL（死亡フラグ）が欠損値。重要な患者背景情報が欠落しており、データ品質に影響があります。
    * 問題点の原因（推測）: データ入力時の欠落、データ収集プロセスの問題
    * 対応策（提案）: 医療機関にIC同意取得日、死亡の有無、死亡日のデータ欠損理由を確認し、データ修正を依頼。

* 問題No.: 9
    * 変数名と値: LB.LBBLFL, QS.QSBLFL (欠損値)
    * 矛盾の内容: LBドメイン、QSドメインにおいて、Baseline Flag（LBBLFL, QSBLFL）が欠損値となっているレコードが散見される。Baseline評価の有無やデータ処理に影響する可能性があります。
    * 問題点の原因（推測）: データ入力時の欠落、Baseline Flag評価未実施
    * 対応策（提案）: 医療機関にBaseline Flag欠損理由を確認し、データ修正を依頼。Define.xmlでLBBLFL, QSBLFLの必須性を確認。

* 問題No.: 10
    * 変数名と値: LB.VISIT = UNSCHEDULED 1.1 (複数レコード)
    * 矛盾の内容: LBドメインにUNSCHEDULED VISITのレコードが複数回記録されている。計画外の検査が頻繁に行われたか、データ入力の誤りの可能性があります。
    * 問題点の原因（推測）: 予定外検査の頻繁な実施、データ入力エラー
    * 対応策（提案）: 医療機関にUNSCHEDULED VISIT複数回記録の理由、計画外検査実施の妥当性を確認。

* 問題No.: 11
    * 変数名と値: LB.LBORNRLO (dataType: string), LB.LBORNRHI (dataType: string), LB.LBSTNRLO (dataType: integer), LB.LBSTNRHI (dataType: integer) (Define.xml定義), JSONデータはすべてstring型
    * 矛盾の内容: Define.xmlとJSONデータ間でLBドメインの一部のデータ型定義が不一致。データ型の不整合は、データ処理や解析に影響を与える可能性があります。
    * 問題点の原因（推測）: Define.xmlのデータ型定義の誤り、JSONデータ作成時のデータ型誤り
    * 対応策（提案）: Define.xmlのLB.LBSTNRLO, LB.LBSTNRHIのデータ型定義をstring型に修正する。

クエリ:
* 患者ID: 01-703-1042
    * クエリNo.: 1
        * 臨床試験結果への影響度合い: Critical
        * 変数名と値: LB.LBTESTCD=ALT, LB.LBORRES=135 (SCREENING 1)
        * 医療機関への問い合わせ文面:
            治験にご協力いただきありがとうございます。
            治験参加者ID: 01-703-1042 のスクリーニング検査（VISIT 1）において、ALT値が基準範囲上限を超過しております。
            ALT値高値の原因、医学的妥当性、再検査の要否についてご教示ください。
            Medical History、Concomitant MedicationsにALT値上昇に関連する情報がないか、肝機能関連の他の検査値も

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.048136
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.385557
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.304933
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T:0.3) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T:0.5) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T:0.7) gemini-2.0-flash-thinking-exp 
------
Event: node_f

...

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.039336
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.193944
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.118984
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T:0.3) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T:0.5) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T:0.7) gemini-2.0-flash-thinking-exp 
------
Event: node_f

はい、承知いたしました。5人のアシスタントの回答を統合し、以下の通りMarkdown形式で出力します。

全体的なデータ品質の評価:
* 総合評価: 一部問題あり
* データクリーニング/再調査が必要な項目: CM.CMENDTC, CM.CMDTC, CM.CMSTDTC, AE.AESTDTC, DS.DSDTC

問題点:
* 問題No.: 1
    * 変数名と値: CM.CMDTCとCM.CMENDTCの日付矛盾 (例: CM.CMDTC=2013-06-03, CM.CMENDTC=2013-05-22)
    * 矛盾の内容: データ収集日 (CMDTC) が併用薬終了日 (CMENDTC) よりも後の日付になっているレコードが存在する。日付の論理矛盾が疑われる。
    * 問題点の原因（推測）: データ入力時の誤り、日付項目の取り違え
    * 対応策（提案）: 医療機関に確認し、データ修正を依頼。

* 問題No.: 2
    * 変数名と値: CMドメインの日付形式 (CMSTDTC, CMENDTC, CMDTC)
    * 矛盾の内容: CMドメインの日付変数の形式がYYYY形式（年のみ）や不正な日付形式で記録されており、Define.xmlの定義dataType="date"と不整合。SDTM IGでは日付変数の形式はISO 8601形式（YYYY-MM-DD）が推奨されている。
    * 問題点の原因（推測）: データ入力時の形式不備、データ変換エラー
    * 対応策（提案）: CMドメインの日付形式をYYYY-MM-DD形式に修正。不明な日付は欠損値 (null) として扱うことを検討。

* 問題No.: 3
    * 変数名と値: AE.AESTDTCとEXドメインの曝露期間の矛盾
    * 矛盾の内容: AEドメインの有害事象開始日 (AESTDTC) がEXドメインの治験薬曝露期間外（前または後）に記録されているレコードが存在する。治験薬との因果関係評価 (AEREL) の妥当性に疑義が生じる。
    * 問題点の原因（推測）: 有害事象と治験薬の因果関係評価の誤り、データ入力時の日付誤り、治験薬投与終了後の遅発性有害事象の可能性
    * 対応策（提案）: 医療機関にAE開始日とEX曝露期間の矛盾について確認し、医学的妥当性を検証。データ入力誤りの場合は修正を依頼。

* 問題No.: 4
    * 変数名と値: CM.CMENDTC (空欄)
    * 矛盾の内容: 多くのCMレコードで併用薬終了日 (CMENDTC) が空欄となっている。データ品質の観点から可能な限り終了日を記録することが望ましい。
    * 問題点の原因（推測）: データ入力時の未入力、併用薬が継続中の可能性
    * 対応策（提案）: 医療機関にCMENDTC空欄の理由を確認し、可能な場合は終了日を追記依頼。併用薬継続中の場合はその旨を記録。

* 問題No.: 5
    * 変数名と値: DS.DSDTCの日付形式混在
    * 矛盾の内容: DS.DSDTCの値の形式がdate形式とdatetime形式で混在。データ形式の不統一。
    * 問題点の原因（推測）: データ入力時の不統一
    * 対応策（提案）: DS.DSDTCのデータ形式をdatetime形式に統一。

クエリ:
* 患者ID: 01-701-1146
    * クエリNo.: 1
        * 臨床試験結果への影響度合い: Critical
        * 変数名と値: CMドメインの日付データ (CMDTC, CMSTDTC, CMENDTC)
        * 医療機関への問い合わせ文面: 患者ID 01-701-1146 の併用薬 (CM) データの日付 (CMDTC, CMSTDTC, CMENDTC) について、記録内容に誤りがないかご確認ください。特に、2008年、1966年、1987年、1942年など、試験期間 (2013年) と矛盾する年が散見されます。記録に誤りがある場合は、正しい日付への修正をお願いいたします。
        * 判断理由: データの整合性に関わる重要な問題であり、誤った日付データはデータ分析結果に影響を与える可能性があるため。特にCMドメインの日付誤りは、EXドメインやAEドメインとのクロスドメイン整合性にも影響を及ぼす可能性がある。

    * クエリNo.: 2
        * 臨床試験結果への影響度合い: Major
        * 変数名と値: AE.AEREL, AE.AESTDTC, EX.EXSTDTC
        * 医療機関への問い合わせ文面: 治験薬と有害事象の因果関係について確認させてください。AEドメインのAE No.1, 2, 3（AETERM: RASH, APPLICATION SITE IRRITATION）の発現日が、EXドメインの治験薬投与開始日よりも前の日付、または治験薬投与期間外で記録されています。にもかかわらず、治験薬との因果関係がPROBABLE（AE No.3）またはPOSSIBLE（AE No.4）と評価されている理由について、医学的な根拠をご教示いただけますでしょうか。
        * 判断理由: 治験薬と有害事象の因果関係は、臨床試験の評価において重要な要素であり、データの信頼性を確保するために医療機関への確認が必要です。

    * クエリNo.: 3
        * 臨床試験結果への影響度合い: Minor
        * 変数名と値: CM.CMENDTC (全レコード)
        * 医療機関への問い合わせ文面: 患者ID 01-701-1146 の併用薬 (CM) データについて、CMENDTC (併用薬終了日) が空欄となっているレコードが複数あります。可能な範囲で結構ですので、空欄となっている理由 (例: 併用薬継続中、データ未入力など) をご教示いただけますでしょうか。もし終了日が判明する場合は、追記のご協力をお願いいたします。
        * 判断理由: データ品質の向上。CMENDTCは必須項目ではないものの、可能な限り記録することが望ましい。

    * クエリNo.: 4
        * 臨床試験結果への影響度合い: Minor
        * 変数名と値: DS.DTC, DS.DSDTC
        * 医療機関への問い合わせ文面: DSドメインのDSDTCにおいて、データ形式がdate形式とdatetime形式で混在しています。データ形式はdatetime形式に統一してもよろしいでしょうか？
        * 判断理由: データ形式の不統一はデータレビューの効率を低下させる可能性があるため、統一性を確認する。

Define.xmlの修正候補:
* ValueList.LB.LBCAT.CHEMISTRY.LBTESTCD (ValueList OID="ValueList.LB.LBCAT.CHEMISTRY.LBTESTCD")
    * ALP (Alkaline Phosphatase) のValueListDefまたはCodeListに基準値範囲に関する定義を追加。（例：RangeCheck要素の追加、CodeListのDecode要素に基準値範囲を追記）

以上、統合結果となります。クエリ発行とデータ修正、Define.xml修正について、引き続き対応をご検討ください。

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.037396
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.190377
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.095809
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T:0.3) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T:0.5) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T:0.7) gemini-2.0-flash-thinking-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T:0.7) gemini-2.0-flash-thinking-exp 
------
Event: node_f

患者ID: 01-701-1146
    逸脱No.: 1
        臨床試験結果への影響度合い: Major
        変数名と値: CM.CMTRT=PAROXETINE, CM.CMDECOD=UNCODED
        逸脱内容: 併用禁止薬の疑義: 患者「01-701-1146」は、WEEK 2でパロキセチン（PAROXETINE）をConcomitant Medicationとして使用しています。パロキセチンは選択的セロトニン再取り込み阻害薬（SSRI）に分類され、プロトコルで併用が禁止されている抗うつ薬に該当する可能性があります。CMDECODがUNCODEDであるため、標準化された薬剤名を確認する必要があります。
        プロトコル該当箇所: 3.8 Concomitant Therapy, 3.4.2.2 Exclusion Criteria [31b]
        判断理由: プロトコル3.4.2.2 Exclusion Criteria [31b]では、抗うつ薬は併用禁止薬としてリストアップされています。パロキセチンが抗うつ薬に該当する場合、プロトコル逸脱となる可能性があります。また、CMDECODがUNCODEDであり、薬剤名の標準化された情報が不足しているため、医療機関への確認が必要と判断しました。

患者ID: 01-701-1146
    クエリNo.: 1
        臨床試験結果への影響度合い: Major
        変数名と値: CM.CMTRT=PAROXETINE, CM.CMDECOD=UNCODED, CM.CMSTDTC, CM.CMDTC
        医療機関への問い合わせ文面:
            治験薬併用状況について確認させてください。
            患者ID: 01-701-1146のWEEK 2のConcomitant Medicationとして報告されているPAROXETINE (パロキセチン) について、以下の点をご確認いただけますでしょうか。

            1.  PAROXETINE (パロキセチン) は、選択・除外基準書あるいは治験実施計画書で規定されている併用禁止薬に該当しますでしょうか。
            2.  CMDECODがUNCODEDとなっていますが、標準化された薬剤名、投与量、投与期間、投与理由についてご教示ください。
            3.  CMドメインのデータにおいて、PAROXETINE (CMSEQ=29) の開始日 (CMSTDTC: 2013-05-21) がデータ収集日 (CMDTC: 2013-06-03) より後になっています。データ入力時の誤りの可能性が考えられますので、記録の確認と修正をお願いします。併用薬の開始日とデータ収集日について、プロトコルで規定されている事項があればご教示ください。

            ご回答よろしくお願いいたします。
        判断理由: PAROXETINE (パロキセチン) が併用禁止薬に該当するか否か、CMDECODの詳細情報、CMSTDTCとCMDTCの日付の矛盾について医療機関に確認が必要と判断しました。これらの情報はプロトコル遵守状況を評価し、データの正確性を確保するために重要です。

,Subject,Task1,Task2,Task3
0,01-703-1042,患者ID: 01-703-1042\n* 2012年12月27日 (Day -65): \n...,全体的なデータ品質の評価:\n* 総合評価: 一部問題あり\n* データクリーニング/再調査...,患者ID: 01-703-1042\n 逸脱No.: 1\n 臨床試験結...
1,01-701-1146,**症例サマリー:**\n\n患者ID: 01-701-1146\n* 2013年05月07...,はい、承知いたしました。5人のアシスタントの回答を統合し、以下の通りMarkdown形式で出...,患者ID: 01-701-1146\n 逸脱No.: 1\n 臨床試験結...


In [16]:
def dataframe_to_text(df: pd.DataFrame) -> str:
    """
    DataFrameを指定されたテキスト形式に変換します。

    Args:
        df: 変換するDataFrame。カラム名は 'Subject', 'Task1', 'Task2', 'Task3' である必要があります。

    Returns:
        変換後のテキストデータ。
    """
    text_data = ""
    for index, row in df.iterrows():
        subject = row['Subject']
        task1 = row['Task1']
        task2 = row['Task2']
        task3 = row['Task3']

        text_data += f"# {subject}\n"
        text_data += f"## Task1\n"
        text_data += f"{task1}\n"
        text_data += f"## Task2\n"
        text_data += f"{task2}\n"
        text_data += f"## Task3\n"
        text_data += f"{task3}\n\n"

    return text_data
output_text = dataframe_to_text(df_results)

In [17]:
# mdファイルに保存
output_file = 'output.md'  # 保存するファイル名を指定
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_text)

print(output_text)

# 01-703-1042
## Task1
患者ID: 01-703-1042
* 2012年12月27日 (Day -65): 
    * ALT (アラニンアミノトランスフェラーゼ) が基準値上限を超えています(135 U/L [基準値上限: 43 U/L])。
    * AST (アスパラギン酸アミノトランスフェラーゼ) が基準値上限を超えています(145 U/L [基準値上限: 36 U/L])。
    * Anisocytes (異形赤血球) が認められました (1 NO UNITS [基準値: 正常範囲内])。
* 2013年02月21日 (Day -9): ナトリウム (Sodium) が基準値下限を下回っています(133 mEq/L [基準値下限: 135 mEq/L])。
* 2013年03月04日 (Day 3): 下痢（軽度）を発症しました。（治験薬との関連性：POSSIBLE、転帰：RECOVERED/RESOLVED）
* 2013年03月05日 (Day 4): 不眠症（軽度）を発症しました。（治験薬との関連性：REMOTE、転帰：RECOVERED/RESOLVED）、下痢 改善
* 2013年03月28日 (Day 27): Ery. Mean Corpuscular Volume (平均赤血球容積, MCV) が基準値上限を超えています(101 fL [基準値上限: 100 fL])。
* 2013年08月31日 (Day 183): AST (アスパラギン酸アミノトランスフェラーゼ) が基準値上限を超えています(38 U/L [基準値上限: 36 U/L])。

---
患者ID: 01-703-1042
* クエリNo.: 1
    * 臨床試験結果への影響度合い: Major
    * 変数名と値: LB.LBTESTCD=ALT, LBORRES=135 U/L (Day -65), LB.LBTESTCD=AST, LBORRES=145 U/L (Day -65)
    * 医療機関への問い合わせ文面: スクリーニング時のALT値およびAST値高値の原因について、医療機関に確認してください。治験薬投与の可否に影響を与える可能性があります。
    * 判断理由: スクリーニング検査時のALT値およびAST値